Author: Jay Qi ([@jayqi](https://github.com/jayqi/))

Demonstrates the functions in [`src/create_points.py`](../src/create_points.py).

**Data setup:** Download `Three_Rivers_Michigan_USA_shp_uid_ae45a8ad-f097-4fd3-bec9-31a366f92657 2.zip` from the project Google Drive and unzip it to `data/raw/`

In [1]:
from pathlib import Path

import geopandas as gpd

from src.create_points import remove_highways, create_points

In [2]:
proj_root = Path().resolve().parent
proj_root

PosixPath('/Users/jqi/repos/street-view-green-view')

In [3]:
shpfile_dir = (
    proj_root
    / "data"
    / "raw"
    / "Three_Rivers_Michigan_USA_shp_uid_ae45a8ad-f097-4fd3-bec9-31a366f92657 2"
)
shpfile_dir

PosixPath('/Users/jqi/repos/street-view-green-view/data/raw/Three_Rivers_Michigan_USA_shp_uid_ae45a8ad-f097-4fd3-bec9-31a366f92657 2')

In [4]:
gdf = gpd.read_file(shpfile_dir)
print(gdf.shape)
gdf.head()

(1203, 12)


,osm_id,osm_type,width,name,highway,layer,oneway,surface,tunnel,bridge,smoothness,geometry
0,628771085,ways_line,None,None,service,None,yes,asphalt,None,None,None,"LINESTRING (-85.65577 41.94215, -85.65435 41.9..."
1,628771094,ways_line,None,None,service,None,None,asphalt,None,None,None,"LINESTRING (-85.65436 41.94281, -85.65436 41.9..."
2,764612663,ways_line,None,None,service,None,None,None,None,None,None,"LINESTRING (-85.65178 41.94118, -85.65195 41.9..."
3,628771076,ways_line,None,None,service,None,yes,asphalt,None,None,None,"LINESTRING (-85.65612 41.94256, -85.65577 41.9..."
4,628771077,ways_line,None,None,service,None,None,asphalt,None,None,None,"LINESTRING (-85.65623 41.94256, -85.65612 41.9..."


In [5]:
# Remove highways

print("Before:", gdf.shape)
print("Before:", sorted(gdf["highway"].unique()))
gdf = remove_highways(gdf)
print("After:", gdf.shape)
print("After:", sorted(gdf["highway"].unique()))

Before: (1203, 12)
Before: ['footway', 'path', 'primary', 'residential', 'secondary', 'service', 'tertiary', 'tertiary_link', 'track', 'trunk', 'trunk_link', 'unclassified']
After: (242, 12)
After: ['path', 'residential', 'track', 'unclassified']


In [6]:
# Visualize the data with folium
gdf.explore(color="purple")

In [7]:
# Use create_points to interpolate points along the roads
print("Roads:", gdf.shape)
points_gdf = create_points(gdf)
print("Points:", points_gdf.shape)
points_gdf.head()

Roads: (242, 12)
Points: (4960, 2)


,osm_id,geometry
0,17967640,POINT (-85.65161 41.95364)
1,17967640,POINT (-85.65155 41.95379)
2,17967640,POINT (-85.65149 41.95394)
3,17967640,POINT (-85.65142 41.95408)
4,17964573,POINT (-85.65606 41.94811)


In [8]:
# Get longest road row
longest_road_osm_id = gdf.loc[gdf["geometry"].to_crs("EPSG:3857").length.idxmax()][
    "osm_id"
]
gdf.loc[gdf["osm_id"] == longest_road_osm_id]

,osm_id,osm_type,width,name,highway,layer,oneway,surface,tunnel,bridge,smoothness,geometry
256,203839666,ways_line,None,Hov Aire Drive,residential,None,None,None,None,None,None,"LINESTRING (-85.61024 41.95477, -85.61025 41.9..."


In [9]:
# Plot with folium
m = gdf[gdf["osm_id"] == longest_road_osm_id].explore(
    color="purple",
)
points_gdf[points_gdf["osm_id"] == longest_road_osm_id].explore(
    m=m,
    color="red",
    marker_kwds=dict(radius=2),
)